Import libraries

In [1]:
import numpy as np
import pandas as pd

import mcdm_functions

Read in and clean data

In [2]:
# Read data
tr_data = pd.read_csv('tr_supplier_data.csv')


# Filter NAICS codes
relevant_naics = [484230]
tr_data = tr_data[tr_data['Primary NAICS'].isin(relevant_naics)]


# Compute scores for opening on weekends
nonopen_values = [np.nan, 'Closed']

saturday_closed_mask = tr_data['Saturday Open'].isin(nonopen_values)
tr_data['Open Saturday'] = 1
tr_data.loc[saturday_closed_mask, 'Open Saturday'] = 0.1

sunday_closed_mask = tr_data['Sunday Open'].isin(nonopen_values)
tr_data['Open Sunday'] = 1
tr_data.loc[sunday_closed_mask, 'Open Sunday'] = 0.1


# Compute scores for credit rating
credit_score_mapper = {'A+': 1,
                       'A': 0.95,
                       'B+': 0.7,
                       'B': 0.5,
                       'C': 0,
                       'C+': 0}

tr_data['Credit Score Num'] = tr_data['Credit Score Alpha'].map(credit_score_mapper)


# Convert sales volume to number
tr_data['Location Sales Volume Actual'] = tr_data['Location Sales Volume Actual'].str.replace('$', '')
tr_data['Location Sales Volume Actual'] = tr_data['Location Sales Volume Actual'].str.replace(',', '')
tr_data['Location Sales Volume Actual'] = pd.to_numeric(tr_data['Location Sales Volume Actual'])


# Compute scores for location
agg_columns = ['City', 'State', 'ZIP Code']
agg_dict = {'Company Name': 'count'}

city_state_pairs = tr_data.groupby(agg_columns).agg(agg_dict).index.tolist()

zip_code_mapper = {}
for city, state, zip_code in city_state_pairs:
    if ((city == 'Tuscaloosa') and (state == 'AL')):
        zip_code_mapper[zip_code] = 1
    elif state == 'AL':
        zip_code_mapper[zip_code] = 0.8
    else:
        zip_code_mapper[zip_code] = 0.5
        
tr_data['Zip Code Score'] = tr_data['ZIP Code'].map(zip_code_mapper)


# Remove unused columns
columns_to_keep = ['Company Name', 
                   'City', 
                   'State', 
                   'ZIP Code',
                   'Open Saturday',
                   'Open Sunday', 
                   'Credit Score Num', 
                   'Zip Code Score', 
                   'Location Sales Volume Actual']
clean_data = tr_data[columns_to_keep].copy()
clean_data = clean_data.reset_index(drop = True)


# Normalize sales volume
sales_volume_array = clean_data['Location Sales Volume Actual'].values
clean_data['Location Sales Volume Actual'] = mcdm_functions.normalize_array(sales_volume_array)


# Define and normalize weights
weights = {'Open Saturday': 3,
           'Open Sunday': 2, 
           'Credit Score Num': 9, 
           'Zip Code Score': 7,
           'Location Sales Volume Actual': 4}

weights = mcdm_functions.normalize_weights_dictionary(weights)

Create MCDM instance

In [3]:
instance = mcdm_functions.mcdm_instance(clean_data, weights)
instance.data

,Company Name,City,State,ZIP Code,Open Saturday,Open Sunday,Credit Score Num,Zip Code Score,Location Sales Volume Actual,WS,WP,TOPSIS
0,AAT Carriers,Jasper,AL,35501,0.1,0.1,0.50,0.8,0.015205,0.426433,0.236380,0.162454
1,Alabama Motor Express,Ashford,AL,36312,0.1,0.1,0.95,0.8,1.000000,0.746000,0.581896,0.701348
2,All Pro Inc,Northport,AL,35476,0.1,0.1,0.70,0.8,0.036423,0.501828,0.306846,0.213414
3,American Way LLC,Cottondale,AL,35453,0.1,0.1,0.00,0.8,0.007287,0.245166,0.000000,0.063574
4,Baggetts Trucking,Piedmont,AL,36272,0.1,0.1,0.00,0.8,0.002350,0.244376,0.000000,0.063131
...,...,...,...,...,...,...,...,...,...,...,...,...
144,"Watson, Tony",Franklin,TN,37064,0.1,0.1,0.50,0.5,0.014443,0.342311,0.205535,0.151966
145,West Tn Transportation Inc,Germantown,TN,38138,0.1,0.1,0.50,0.5,0.023739,0.343798,0.222544,0.153508
146,Worldwide Freight Corp,Chattanooga,TN,37421,0.1,0.1,0.50,0.5,0.034230,0.345477,0.235964,0.155600
147,Xcricket Sales,Murfreesboro,TN,37129,0.1,0.1,0.00,0.5,0.002784,0.160445,0.000000,0.002369


Get the top five providers by ech ranking method.

In [4]:
instance.data.nlargest(5, columns = 'WS')

,Company Name,City,State,ZIP Code,Open Saturday,Open Sunday,Credit Score Num,Zip Code Score,Location Sales Volume Actual,WS,WP,TOPSIS
1,Alabama Motor Express,Ashford,AL,36312,0.1,0.1,0.95,0.8,1.000000,0.746000,0.581896,0.701348
74,First Fleet Inc,Murfreesboro,TN,37129,1.0,1.0,0.95,0.5,0.180237,0.710838,0.614649,0.425982
92,JB Hunt Transport Svc Inc,Memphis,TN,38118,1.0,1.0,0.95,0.5,0.035596,0.687695,0.474150,0.377515
31,My Way Transportation Inc,Hanceville,AL,35077,1.0,0.1,0.70,0.8,0.033140,0.609302,0.398435,0.313983
15,Century Carriers,Cottondale,AL,35453,0.1,0.1,0.95,0.8,0.121401,0.605424,0.415261,0.289724


In [5]:
instance.data.nlargest(5, columns = 'WP')

,Company Name,City,State,ZIP Code,Open Saturday,Open Sunday,Credit Score Num,Zip Code Score,Location Sales Volume Actual,WS,WP,TOPSIS
74,First Fleet Inc,Murfreesboro,TN,37129,1.0,1.0,0.95,0.5,0.180237,0.710838,0.614649,0.425982
1,Alabama Motor Express,Ashford,AL,36312,0.1,0.1,0.95,0.8,1.000000,0.746000,0.581896,0.701348
92,JB Hunt Transport Svc Inc,Memphis,TN,38118,1.0,1.0,0.95,0.5,0.035596,0.687695,0.474150,0.377515
143,W E Graham Inc,Memphis,TN,38118,0.1,0.1,0.95,0.5,0.299576,0.549932,0.420662,0.363154
15,Century Carriers,Cottondale,AL,35453,0.1,0.1,0.95,0.8,0.121401,0.605424,0.415261,0.289724


In [6]:
instance.data.nlargest(5, columns = 'TOPSIS')

,Company Name,City,State,ZIP Code,Open Saturday,Open Sunday,Credit Score Num,Zip Code Score,Location Sales Volume Actual,WS,WP,TOPSIS
1,Alabama Motor Express,Ashford,AL,36312,0.1,0.1,0.95,0.8,1.000000,0.746000,0.581896,0.701348
74,First Fleet Inc,Murfreesboro,TN,37129,1.0,1.0,0.95,0.5,0.180237,0.710838,0.614649,0.425982
92,JB Hunt Transport Svc Inc,Memphis,TN,38118,1.0,1.0,0.95,0.5,0.035596,0.687695,0.474150,0.377515
143,W E Graham Inc,Memphis,TN,38118,0.1,0.1,0.95,0.5,0.299576,0.549932,0.420662,0.363154
31,My Way Transportation Inc,Hanceville,AL,35077,1.0,0.1,0.70,0.8,0.033140,0.609302,0.398435,0.313983
